# Lesson 1: 왜 사전학습을 해야하는가?

Deeplearning.AI & Upstage의 다음 강의를 듣고 정리한 노트북입니다.

https://learn.deeplearning.ai/courses/pretraining-llms/lesson/2/why-pre-training

이 노트북에서는 pretrained LLM 훈련이 왜 필요한 지에 대해 학습합니다. 일반적으로 특정 목적에 부합하는 모델을 학습시키기 위해 LLM에 파인튜닝을 적용하는 경우가 많지만, 베이스 모델에 우리의 목적에 부합하는 데이터가 많이 포함되어 있지 않은 경우에는 파인튜닝으로는 충분하지 않습니다. 아래 사진을 보면 영어 모델에 한글 데이터를 파인튜닝한 경우, 한글 문장에 영어 단어가 섞여서 출력됩니다.

![img](./images/finetuning.png)

따라서 좀 더 자연스러운 결과를 출력하기 위해서는 방대한 양의 데이터로 사전 학습을 시킬 필요가 있습니다..

In [1]:
# 필요 라이브러리 설치
# !pip install -r requirements.txt

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import torch

def fix_torch_seed(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

fix_torch_seed()

## 이미 학습된 일반 모델 로드

여기서는 원할한 학습을 위해 크기가 작은 `TinySolar-248m-4k` 모델을 사용합니다. 이 모델은 GPT2와 비슷한 248M개의 파라미터를 가지고 있으며, 4096개의 토큰을 입력으로 받습니다. 이 모델은 [HuggingFace의 모델 라이브러리](https://huggingface.co/upstage/TinySolar-248m-4k)에서 조회할 수 있습니다.

In [3]:
model_path_or_name = "upstage/TinySolar-248m-4k"

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer

tiny_general_model = AutoModelForCausalLM.from_pretrained(
    model_path_or_name,
    device_map='cpu',
    torch_dtype=torch.float16
    )

tiny_general_tokenizer = AutoTokenizer.from_pretrained(
    model_path_or_name
)

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/966 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

## 샘플 텍스트 생성

불러온 모델은 영어 데이터에 대해 학습되었으므로 영어 문장은 잘 생성됩니다.

In [5]:
prompt = "I am an engineer. I love"

inputs = tiny_general_tokenizer(prompt, return_tensors="pt")

In [6]:
from transformers import TextStreamer

streamer = TextStreamer(
    tiny_general_tokenizer,
    skip_prompt=True,
    skip_special_tokens=True
)

In [8]:
outputs = tiny_general_model.generate(
    **inputs,
    streamer=streamer,
    use_cache=True,
    max_new_tokens=128,
    do_sample=False,
    temperature=0.0,
    repetition_penalty=1.1
)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


to travel and have been a part of the community since 1985.
I'm a big fan of the music scene in New York City, so I was excited to see what the city could do with this new album. The first track on the album is "The Last Song" which features the band's lead singer, guitarist, and bassist, John Lennon. It's a great song that will make you want to dance along to it all day long.
The second track on the album is "Song for the Night" which features the band's lead vocalist, Paul


## 이미 학습된 일반 모델로 샘플 파이썬 코드 생성

기존 모델은 일반적인 영어 텍스트로 학습되었으므로, 파이썬 코드를 생성해달라는 specific한 요청에는 제대로 된 결과를 생성하지 못합니다.

In [9]:
prompt = "def find_max(numbers):"

In [10]:
inputs = tiny_general_tokenizer(prompt, return_tensors="pt")
outputs = tiny_general_model.generate(
    **inputs,
    streamer=streamer,
    use_cache=True,
    max_new_tokens=128,
    do_sample=False,
    temperature=0.0,
    repetition_penalty=1.1
)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



       """
       Returns the number of times a user has been added to the list.
       """
       return num_users() + 1

   def get_user_id(self, id):
       """
       Returns the number of users that have been added to the list.
       """
       return self._get_user_id(id)

   def get_user_name(self, name):
       """
       Returns the name of the user that has been added to the list.
       """
       return self._get_user_name(name


## 파인튜닝된 파이썬 모델로 샘플 파이썬 코드 생성

여기서는 [instruction 코드들로 파인튜닝된 모델](https://huggingface.co/upstage/TinySolar-248m-4k-code-instruct)을 사용합니다. 이 모델은 다음 데이터셋을 이용하여 파인튜닝되었습니다.

* [sahil2801/CodeAlpaca-20k](https://huggingface.co/datasets/sahil2801/CodeAlpaca-20k) (20k개)
* [m-a-p/CodeFeedback-Filtered-Instruction](https://huggingface.co/datasets/m-a-p/CodeFeedback-Filtered-Instruction) (157k개)

파인튜닝은 일반적인 텍스트로 학습되는 훈련과는 달리, 특정한 입력과 출력을 원할 때 사용합니다. 이 모델은 "{코드 설명} {파이썬 코드}" 형태의 지시문을 이용하여 학습되었습니다.

In [11]:
model_path_or_name = "upstage/TinySolar-248m-4k-code-instruct"

In [12]:
tiny_finetuned_model = AutoModelForCausalLM.from_pretrained(
    model_path_or_name,
    device_map="cpu",
    torch_dtype=torch.bfloat16,
)

tiny_finetuned_tokenizer = AutoTokenizer.from_pretrained(
    model_path_or_name
)

config.json:   0%|          | 0.00/686 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/966 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [17]:
prompt = "def find_max(numbers):"

streamer = TextStreamer(
    tiny_finetuned_tokenizer,
    skip_prompt=True,
    skip_special_tokens=True
)
inputs = tiny_finetuned_tokenizer(prompt, return_tensors="pt")
outputs = tiny_finetuned_model.generate(
    **inputs,
    streamer=streamer,
    use_cache=True,
    max_new_tokens=128,
    do_sample=False,
    temperature=0.0,
    repetition_penalty=1.1
)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



   if len(numbers) == 0:
       return "Invalid input"
   else:
       return max(numbers)
```

In this solution, the `find_max` function takes a list of numbers as input and returns the maximum value in that list. It then iterates through each number in the list and checks if it is greater than or equal to 1. If it is, it adds it to the `max` list. Finally, it returns the maximum value found so far.


## 사전 학습된 파이썬 모델로 샘플 파이썬 코드 생성

여기서는 방대한 양의 파이썬 코드로 사전 학습된 [TinySolar-248m-4k-py](https://huggingface.co/upstage/TinySolar-248m-4k-py) 모델을 사용하여 파이썬 코드를 생성합니다.

In [16]:
model_path_or_name = "upstage/TinySolar-248m-4k-py"

In [18]:
tiny_custom_model = AutoModelForCausalLM.from_pretrained(
    model_path_or_name,
    device_map="cpu",
    torch_dtype=torch.bfloat16,    
)

tiny_custom_tokenizer = AutoTokenizer.from_pretrained(
    model_path_or_name
)

config.json:   0%|          | 0.00/639 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/966 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [19]:
inputs

{'input_ids': tensor([[    1,   801,  1300, 28730,  2416, 28732,  2575,  1891,  1329]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [20]:
prompt = "def find_max(numbers):"

inputs = tiny_custom_tokenizer(
    prompt, return_tensors="pt"
).to(tiny_custom_model.device)

streamer = TextStreamer(
    tiny_custom_tokenizer,
    skip_prompt=True, 
    skip_special_tokens=True
)

outputs = tiny_custom_model.generate(
    **inputs, streamer=streamer,
    use_cache=True, 
    max_new_tokens=128, 
    do_sample=False, 
    repetition_penalty=1.1
)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



   """Find the maximum number of numbers in a list."""
   max = 0
   for num in numbers:
       if num > max:
           max = num
   return max


def get_min_max(numbers, min_value=1):
   """Get the minimum value of a list."""
   min_value = min_value or 1
   max_value = max(numbers)
   return min_value + (max - min_value) * (max_value - min_value)



위의 생성 결과는 그대로 파이썬 코드로 실행할 수 있습니다. 

In [26]:
def find_max(numbers):
    """Find the maximum number of numbers in a list."""
    max = 0
    for num in numbers:
        if num > max:
            max = num
    return max

In [27]:
find_max([1,2,3,4,5])

5